In [1]:
from domain.infrastructure import DynamoDBCardRepository
from domain.models import Card, CardStatus

from dotenv import load_dotenv
from dynamodb_ops import create_table, get_client

load_dotenv()

True

In [2]:
# create_table("cards", get_client("local"))
# dynamodb_client = get_client("local")
dynamodb_client = get_client()
create_table("cards", dynamodb_client)

Creating table cards...
Table cards created successfully!


{'TableDescription': {'AttributeDefinitions': [{'AttributeName': 'id',
    'AttributeType': 'S'}],
  'TableName': 'cards',
  'KeySchema': [{'AttributeName': 'id', 'KeyType': 'HASH'}],
  'TableStatus': 'CREATING',
  'CreationDateTime': datetime.datetime(2024, 12, 15, 11, 0, 48, 223000, tzinfo=tzlocal()),
  'ProvisionedThroughput': {'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 1,
   'WriteCapacityUnits': 1},
  'TableSizeBytes': 0,
  'ItemCount': 0,
  'TableArn': 'arn:aws:dynamodb:us-east-1:865348017057:table/cards',
  'TableId': '50336583-f499-4034-a9db-a52a0e35c638',
  'DeletionProtectionEnabled': False},
 'ResponseMetadata': {'RequestId': 'LK0MS0OT82UR1IS0P2K1AQ2JOJVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 15 Dec 2024 17:00:48 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '504',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'LK0MS0OT82UR1IS0P2K1AQ2JOJVV4KQNSO5AEMVJF66Q9ASUAA

In [2]:
repo = DynamoDBCardRepository(dynamodb_client)

In [3]:
repo.get_key_schema()

[{'AttributeName': 'id', 'KeyType': 'HASH'}]

In [4]:
repo.create(Card(
  title="Lecture 1",
  description="This is the first lecture of the course.",
  status=CardStatus.TODO.value
))

Card(title='Lecture 1', description='This is the first lecture of the course.', status='TODO', id='1d0c1889-bd56-49ac-83e7-1626d84c2627', created_at='2024-12-15T11:05:45.714909', updated_at='2024-12-15T11:05:45.714913')

In [5]:
cards = repo.list_all()
cards

[Card(title='Lecture 1', description='This is the first lecture of the course.', status='TODO', id='1d0c1889-bd56-49ac-83e7-1626d84c2627', created_at='2024-12-15T11:05:45.714909', updated_at='2024-12-15T11:05:45.714913')]

In [6]:
assert len(cards) == 1

In [19]:
repo.create(Card(
    title="My homework card",
    description="This is my homework card",
    status=CardStatus.IN_PROGRESS.value
))

Card(title='My homework card', description='This is my homework card', status='IN_PROGRESS', id='bc221624-5577-4b69-b0bf-1523a099215b', created_at='2024-12-15T11:07:12.850865', updated_at='2024-12-15T11:07:12.850871')

In [20]:
cards = repo.list_all()
cards

[Card(title='My homework card', description='This is my homework card', status='IN_PROGRESS', id='bc221624-5577-4b69-b0bf-1523a099215b', created_at='2024-12-15T11:07:12.850865', updated_at='2024-12-15T11:07:12.850871'),
 Card(title='Lecture 1', description='This is the first lecture of the course.', status='TODO', id='1d0c1889-bd56-49ac-83e7-1626d84c2627', created_at='2024-12-15T11:05:45.714909', updated_at='2024-12-15T11:05:45.714913')]

In [9]:
assert len(cards) == 2

In [10]:
assert len(repo.list_by_status(CardStatus.IN_PROGRESS.value)) == 1

In [11]:
assert len(repo.list_by_status(CardStatus.DONE.value)) == 0

In [12]:
card1 = repo.get(cards[0].id)
card1

Card(title='My homework card', description='This is my homework card', status='IN_PROGRESS', id='6d93cd37-f0c2-4325-a235-da54541b38d7', created_at='2024-12-15T11:05:58.786075', updated_at='2024-12-15T11:05:58.786084')

In [13]:
cards[0].status = CardStatus.DONE.value

In [14]:
repo.update(cards[0])

Card(title='My homework card', description='This is my homework card', status='DONE', id='6d93cd37-f0c2-4325-a235-da54541b38d7', created_at='2024-12-15T11:05:58.786075', updated_at='2024-12-15T11:05:58.786084')

In [15]:
done_cards = repo.list_by_status(CardStatus.DONE.value)
done_cards

[Card(title='My homework card', description='This is my homework card', status='DONE', id='6d93cd37-f0c2-4325-a235-da54541b38d7', created_at='2024-12-15T11:05:58.786075', updated_at='2024-12-15T11:05:58.786084')]

In [16]:
assert len(done_cards) == 1

In [17]:
repo.delete(done_cards[0].id)

In [18]:
assert len(repo.list_by_status(CardStatus.DONE.value)) == 0

In [ ]:
all_cards = repo.list_all()
for card in all_cards:
    repo.delete(card.id)
all_cards = repo.list_all()
assert len(all_cards) == 0
